# settings

In [1]:
import os
import sys
import time
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
import unicodedata
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import json
import pickle
from sklearn.model_selection import train_test_split
%matplotlib inline
from nltk import sent_tokenize

import math
import os
import random
import string

# Pytorch library for training
import torch
from torch import optim

import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from torchtext.data import Field, BucketIterator, Example

#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# pre-trained word embedding by GolVe
https://github.com/stanfordnlp/GloVe
...Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 300d vectors, 822 MB download): glove.6B.zip
Due to the nature of WikiHow dataset, we choose word embedding result file 'glove.6B' which is pre-trained on Wikipedia and Gigaword dataset. Besides, it contains four .text for different embdedding vector length: 50, 100, 200, 300. 

# load WikiHow data

In [2]:
%%time
data = pd.read_csv('.\Dataset\wikihowSep.csv')
data = data[:1000]
data = data.astype(str)
rows, columns = data.shape

Wall time: 13.1 s


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

expand contraction by https://github.com/khurram6968/NLP-Expand-Contraction-Python/blob/master/NLP.py

In [4]:
contraction_map={
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd've": "how did have",
    "how'll": "how will",
    "how's": "how is",
    "I'd": "I would",
    "I'd've": "I would have",
    "I'll": "I will",
    "I'll've": "I will have",
    "I'm": "I am",
    "I've": "I have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "might have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "shall'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "will't've": "will not have",
    "would've": "would have",
    "would't": "would not",
    "would't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you have all",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have",
}

def expand_contractions(sent, mapping):
    #pattern for matching contraction with their expansions
    pattern = re.compile('({})'.format('|'.join(mapping.keys())), flags=re.IGNORECASE|re.DOTALL)
    
    def expand_map(contraction):
        #using group method to access subgroups of the match
        match = contraction.group(0)
        #to retain correct case of the word
        first_char = match[0]
        #find out the expansion
        expansion = mapping.get(match) if mapping.get(match) else mapping.get(match.lower())
        expansion = first_char + expansion[1:]
        return expansion
    #using sub method to replace all contractions with their expansions for a sentence
    #function expand_map will be called for every non overlapping occurence of the pattern
    expand_sent = pattern.sub(expand_map, sent)
    return expand_sent

simple pre-processing by https://towardsdatascience.com/nlp-building-text-cleanup-and-preprocessing-pipeline-eba4095245a0

In [5]:
print(data.shape)

(1000, 5)


discard unnecessary data, due to computational resource limitation

In [6]:
data['num_word'] = data['text'].apply(lambda x: len(str(x).split()))
num_word = np.sort(data['num_word'].values)

data['num_word_hl'] = data['headline'].apply(lambda x: len(str(x).split()))
num_word_hl = np.sort(data['num_word_hl'].values)

min_text_len = num_word[int(len(num_word)*0.1)]
max_text_len = num_word[int(len(num_word)*0.95)]

min_hl_len = num_word_hl[int(len(num_word_hl)*0.1)]

In [7]:
headline_ratio_threshold = 0.75

del_idx = []
for i in range(rows):
#     if data['num_word'][i] < max_text_len and data['num_word'][i] > min_text_len:
    if data['num_word'][i] < min_text_len:
        del_idx.append(i)
    else:
        pass
    
    if max_text_len < data['num_word'][i]:
        del_idx.append(i)
    else:
        pass

    if data['num_word_hl'][i] > headline_ratio_threshold*data['num_word'][i]:
        del_idx.append(i)
    else:
        pass
    
    if data['num_word_hl'][i] < min_hl_len:
        del_idx.append(i)
    else:
        pass


In [8]:
len(del_idx)

331

In [9]:
data_new = data.drop(del_idx)

In [10]:
data_new.shape

(755, 7)

In [11]:
def data_loader(dataframe, target_col): 
    # extraction from dataframe in to list
    text = [article for article in getattr(dataframe, target_col)]
    
    # Removing Accented Characters
    text = [unicodedata.normalize('NFKD', sentence).encode('ascii', 'ignore').decode('utf-8', 'ignore') for sentence in text]
    
    # Expanding Contractions
    text = [expand_contractions(sentence, contraction_map) for sentence in text]

    # Removing Special Characters
    pat1 = r'[^a-zA-z0-9.,!?\s]' 
    # pat1 = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    text = [re.sub(pat1, '', sentence) for sentence in text]
    
    # Removing Extra Commas
    pat2 = r'[.]+[\n]+[,]'
    text = [re.sub(pat2,".\n", sentence) for sentence in text]
    
    # Removing extra whitespaces and tabs
    # pat3 = r'^\s*|\s\s*'
    pat3 = r'^\s+$|\s+$'
    text = [re.sub(pat3, '', sentence).strip() for sentence in text]
    
    # Add space before .
    pat4 = r'\.|\?|\！'
    text = [re.sub(pat4, ' . ', sentence) for sentence in text]
    
    # Lowercase
    text = [sentence.lower() for sentence in text]
    
    # tokenize
    text = [('sos ' + sentence + ' eos').split() for sentence in text]
    
    return np.array(text, dtype=object)

In [12]:
%%time
text_data = data_loader(data_new, 'text')
headline_data = data_loader(data_new, 'headline')

Wall time: 523 ms


In [13]:
len(text_data)

755

train, test, validation split

In [14]:
text_train, text_test, headline_train, headline_test = train_test_split(text_data, headline_data, test_size=0.1, random_state=1)

text_train, text_dev, headline_train, headline_dev = train_test_split(text_train, headline_train, test_size=0.1, random_state=1)

sort sentence from longer to shorter length, for more efficient processing

In [15]:
def data_sorter(text, headline): 
    headline = [y for x,y in sorted(zip(text, headline), key = lambda pair: len(pair[0]), reverse = True)]
    text = list(text)
    text.sort(key = lambda x: len(x), reverse = True)

    return np.array(text), np.array(headline)

In [16]:
text_train, headline_train = data_sorter(text_train, headline_train)
text_test,  headline_test  = data_sorter(text_test, headline_test)
text_dev,   headline_dev   = data_sorter(text_dev, headline_dev)

<ipython-input-15-515ae7e24249>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(text), np.array(headline)


# save data

In [17]:
# train
np.save('./Dataset3/text_train.npy', text_train)
np.save('./Dataset3/headline_train.npy', headline_train)

# dev
np.save('./Dataset3/text_dev.npy', text_dev)
np.save('./Dataset3/headline_dev.npy', headline_dev)

# test
np.save('./Dataset3/text_test.npy', text_test)
np.save('./Dataset3/headline_test.npy', headline_test)

# Build Vocabulary
https://www.kdnuggets.com/2019/11/create-vocabulary-nlp-tasks-python.html

In [18]:
# PAD_token = 0   # Used for padding short sentences
# SOS_token = 1   # Start-of-sentence token
# EOS_token = 2   # End-of-sentence token

In [19]:
class Vocabulary:
    PAD_token = 0   # Used for padding short sentences
    SOS_token = 1   # Start-of-sentence token
    EOS_token = 2   # End-of-sentence token

    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
#         self.index2word = {PAD_token: 'pad', SOS_token: 'sos', EOS_token: 'eos'}
        self.index2word = {0: "pad", 1: "sos", 2: "eos"}
        self.num_words = 3
        self.num_sentences = 0
        self.longest_sentence = 0

    def add_word(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
            
    def add_sentence(self, sentence):
        sentence_len = 0
        for word in sentence.split(' '):
            sentence_len += 1
            self.add_word(word)
        if sentence_len > self.longest_sentence:
            self.longest_sentence = sentence_len
        self.num_sentences += 1

    def to_word(self, index):
        return self.index2word[index]

    def to_index(self, word):
        return self.word2index[word]

#### Padding

In [20]:
def batchpad_data(data):
    voc = Vocabulary('text')
    data_idx = []
    for i in range(len(data)):
        idx = []
        for word in data[i]:
            voc.add_word(word)
    
        for word in data[i]:
            idx.append(voc.to_index(word))

        data_idx.append(torch.tensor(idx))
        
    data_pad = torch.nn.utils.rnn.pad_sequence(data_idx, batch_first=True, padding_value=0.0)
    
    return data_idx, data_pad, voc

In [21]:
## Training padding
text_train_idx, text_train_pad, text_train_voc = batchpad_data(text_train)
headline_train_idx, headline_train_pad, headline_train_voc= batchpad_data(headline_train)

## Validation padding
text_dev_idx, text_dev_pad, text_dev_voc = batchpad_data(text_dev)
headline_dev_idx, headline_dev_pad, headline_dev_voc = batchpad_data(headline_dev)

## Testing padding
text_test_idx, text_test_pad, text_test_voc = batchpad_data(text_test)
headline_test_idx, headline_test_pad, headline_test_voc = batchpad_data(headline_test)

In [22]:
print(text_train_pad.shape)
print(headline_train_pad.shape)
print(len(text_train_voc.word2index.keys()))

torch.Size([611, 207])
torch.Size([611, 52])
7559


In [23]:
text_train[-1]

['sos',
 'studies',
 'have',
 'shown',
 'that',
 'eating',
 'two',
 'servings',
 'of',
 'seafood',
 'every',
 'week',
 'can',
 'help',
 'child',
 'development',
 'and',
 'even',
 'result',
 'in',
 'higher',
 'iqs',
 '.',
 'eos']

In [24]:
headline_train_pad[-1]

tensor([   3,  393, 1206, 1354,   12,   13,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])

In [25]:
## Zip text and headline together for dataloader
traindata = torch.utils.data.TensorDataset(text_train_pad, headline_train_pad)

devdata = torch.utils.data.TensorDataset(text_dev_pad, headline_dev_pad)

testdata = torch.utils.data.TensorDataset(text_test_pad, headline_test_pad)

#### Batch data

In [49]:
## Set batch size and split data after padding to batches
def batch_dataloader(data, Batch_size):
    data_dataloader = torch.utils.data.DataLoader(data, batch_size=Batch_size, shuffle=False, num_workers=0)
#     for i in data_dataloader:
#         i = torch.transpose(i, 0 ,1)
#         print(data_dataloader)
    
    return data_dataloader

In [50]:
## Training data batching
trainloader = batch_dataloader(traindata, 15)

devloader = batch_dataloader(devdata, 2)

testloader = batch_dataloader(testdata, 2)

### Seq2seq

In [51]:
### Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# device = "cpu"

In [52]:
## Transpose dataset for training
text_testrun, hl_testrun = next(iter(trainloader))

text_testrun = torch.transpose(text_testrun, 0 ,1)
hl_testrun = torch.transpose(hl_testrun, 0 ,1)

In [53]:
text_testrun = text_testrun.to(device)
hl_testrun = hl_testrun.to(device)

In [54]:
len(text_train_voc.index2word.keys())

7562

In [55]:
print(text_testrun.shape)
print(hl_testrun.shape)

torch.Size([207, 15])
torch.Size([52, 15])


In [56]:
### Set parameters
# input_size = int(torch.max(text_train_pad) + 1)
# output_size = int(torch.max(headline_train_pad) + 1)

input_size = int(len(text_train_voc.index2word.keys())+1)
output_size = int(len(headline_train_voc.index2word.keys())+1)

enc_emb_size = 256
dec_emb_size = 256
hid_size = 128

n_layers = 2
enc_dropout = 0.5
dec_dropout = 0.5

In [57]:
class Encoder(nn.Module):
    def __init__(self, input_size, emb_size, hid_size, n_layers, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.hid_size = hid_size
        self.input_size = input_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hid_size, n_layers, dropout=dropout)
        
#         self.lstm = nn.LSTM(emb_size, hid_size, n_layers, dropout=dropout, bidirectional = True)
        

    def forward(self, x):
        embedded = self.embedding(x) 
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return hidden, cell

In [58]:
encoder = Encoder(input_size, enc_emb_size, hid_size, n_layers, enc_dropout).to(device)

hidden, cell = encoder(text_testrun)
hidden.shape, cell.shape

(torch.Size([2, 15, 128]), torch.Size([2, 15, 128]))

In [59]:
class Decoder(nn.Module):
    def __init__(self, output_size, emb_size, hid_size, n_layers, dropout):
        super().__init__()
        self.emb_size = emb_size
        self.hid_size = hid_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(output_size, emb_size)
        self.lstm = nn.LSTM(emb_size, hid_size, n_layers, dropout=dropout)
        self.out = nn.Linear(hid_size, output_size, bias = True)

    def forward(self, output, hidden, cell):
        embedded = self.embedding(output.unsqueeze(0))
        
        outputs, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        prediction = self.out(outputs.squeeze(0))
        return prediction, hidden, cell

In [60]:
decoder = Decoder(output_size, dec_emb_size, hid_size, n_layers, dec_dropout).to(device)

prediction, hidden, cell = decoder(hl_testrun[0], hidden, cell)
prediction.shape, hidden.shape, cell.shape

(torch.Size([15, 1414]), torch.Size([2, 15, 128]), torch.Size([2, 15, 128]))

In [61]:
## not yet debugged
class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim, bias=False)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, s, enc_output):
        
        # s = [batch_size, dec_hid_dim]
        # enc_output = [src_len, batch_size, enc_hid_dim * 2]
        
        batch_size = enc_output.shape[1]
        src_len = enc_output.shape[0]
        
        # repeat decoder hidden state src_len times
        # s = [batch_size, src_len, dec_hid_dim]
        # enc_output = [batch_size, src_len, enc_hid_dim * 2]
        s = s.unsqueeze(1).repeat(1, src_len, 1)
        enc_output = enc_output.transpose(0, 1)
        
        # energy = [batch_size, src_len, dec_hid_dim]
        energy = torch.tanh(self.attn(torch.cat((s, enc_output), dim = 2)))
        
        # attention = [batch_size, src_len]
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

#### seq2seq

In [62]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, device: torch.device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, text_batch, headline_batch, teacher_forcing_ratio: float=0.5):
        
#         text_batch = torch.transpose(text_batch, 0, 1)
#         headline_batch = torch.transpose(headline_batch, 0, 1)

        max_len, batch_size = headline_batch.shape
        headline_vocab_size = self.decoder.output_size

        # tensor to store decoder's output
        outputs = torch.zeros(max_len, batch_size, headline_vocab_size).to(self.device)

        # last hidden & cell state of the encoder is used as the decoder's initial hidden state
        hidden, cell = self.encoder(text_batch)

        for i in range(1, max_len):
            prediction, hidden, cell = self.decoder(headline_batch[0], hidden, cell)
            outputs[i] = prediction

            if random.random() < teacher_forcing_ratio:
                hl_batch_i = headline_batch[i]
            else:
                hl_batch_i = prediction.argmax(1)

        return outputs

In [63]:
# note that this implementation assumes that the size of the hidden layer,
# and the number of layer are the same between the encoder and decoder
encoder = Encoder(input_size, enc_emb_size, hid_size, n_layers, enc_dropout)
decoder = Decoder(output_size, dec_emb_size, hid_size, n_layers, dec_dropout)
seq2seq = Seq2Seq(encoder, decoder, device).to(device)
seq2seq

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7563, 256)
    (lstm): LSTM(256, 128, num_layers=2, dropout=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(1414, 256)
    (lstm): LSTM(256, 128, num_layers=2, dropout=0.5)
    (out): Linear(in_features=128, out_features=1414, bias=True)
  )
)

In [64]:
# outputs = seq2seq(next(iter(text_trainloader)), next(iter(headline_trainloader)))
outputs = seq2seq(text_testrun, hl_testrun)
outputs.shape

torch.Size([52, 15, 1414])

In [65]:
optimizer = optim.Adam(seq2seq.parameters())

# ignore the padding = 0
criterion = nn.CrossEntropyLoss(ignore_index = 0)

In [66]:
def train(seq2seq, trainloader, optimizer, criterion):
    seq2seq.train()

    epoch_loss = 0
    for text_batch, hl_batch in trainloader:
        text_batch = torch.transpose(text_batch, 0, 1)
        hl_batch = torch.transpose(hl_batch, 0, 1)
        
        ## send to cuda
        text_batch = text_batch.to(device)
        hl_batch = hl_batch.to(device)
        
        optimizer.zero_grad()
        
        outputs = seq2seq(text_batch, hl_batch)
        outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
        
        hl_flatten = hl_batch[1:].reshape(-1)
        loss = criterion(outputs_flatten, hl_flatten)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(trainloader)

In [67]:
def evaluate(seq2seq, trainloader, criterion):
    seq2seq.eval()

    epoch_loss = 0
    with torch.no_grad():
        for text_batch, hl_batch in trainloader:
            text_batch = torch.transpose(text_batch, 0, 1)
            hl_batch = torch.transpose(hl_batch, 0, 1)
            
            ## send to cuda
            text_batch = text_batch.to(device)
            hl_batch = hl_batch.to(device)
            
            # teacher forcing not used
            outputs = seq2seq(text_batch, hl_batch, teacher_forcing_ratio=0) 
            outputs_flatten = outputs[1:].view(-1, outputs.shape[-1])
            
            hl_flatten = hl_batch[1:].reshape(-1)
            loss = criterion(outputs_flatten, hl_flatten)
            epoch_loss += loss.item()

    return epoch_loss / len(trainloader)

In [68]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [69]:
num_epoch = 20
best_valid_loss = float('inf')

for epoch in range(num_epoch):    
    start_time = time.time()
    train_loss = train(seq2seq, trainloader, optimizer, criterion)
    valid_loss = evaluate(seq2seq, devloader, criterion)
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(seq2seq.state_dict(), 'tut1-model.pt')

    # it's easier to see a change in perplexity between epoch as it's an exponential
    # of the loss, hence the scale of the measure is much bigger
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 7s
	Train Loss: 6.048 | Train PPL: 423.063
	 Val. Loss: 6.752 |  Val. PPL: 855.906
Epoch: 02 | Time: 0m 8s
	Train Loss: 5.315 | Train PPL: 203.432
	 Val. Loss: 6.808 |  Val. PPL: 905.089
Epoch: 03 | Time: 0m 8s
	Train Loss: 5.200 | Train PPL: 181.329
	 Val. Loss: 6.933 |  Val. PPL: 1025.794
Epoch: 04 | Time: 0m 8s
	Train Loss: 5.124 | Train PPL: 168.066
	 Val. Loss: 7.010 |  Val. PPL: 1108.034
Epoch: 05 | Time: 0m 8s
	Train Loss: 5.063 | Train PPL: 158.121
	 Val. Loss: 7.046 |  Val. PPL: 1148.292
Epoch: 06 | Time: 0m 8s
	Train Loss: 5.009 | Train PPL: 149.819
	 Val. Loss: 7.109 |  Val. PPL: 1223.117
Epoch: 07 | Time: 0m 8s
	Train Loss: 4.969 | Train PPL: 143.844
	 Val. Loss: 7.162 |  Val. PPL: 1289.159
Epoch: 08 | Time: 0m 8s
	Train Loss: 4.931 | Train PPL: 138.507
	 Val. Loss: 7.201 |  Val. PPL: 1340.913
Epoch: 09 | Time: 0m 8s
	Train Loss: 4.900 | Train PPL: 134.333
	 Val. Loss: 7.261 |  Val. PPL: 1423.183
Epoch: 10 | Time: 0m 8s
	Train Loss: 4.875 | Train PPL: 1

In [70]:
test = next(iter(testloader))

In [77]:
with torch.no_grad():
    outputs = seq2seq(test[:][0], test[:][1], teacher_forcing_ratio=0)

RuntimeError: Input, output and indices must be on the current device

### BACKUP (ALL CODES BELOW NOT USED)

In [92]:
# class DecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size):
#         super(DecoderRNN, self).__init__()
#         self.hidden_size = hidden_size

#         self.embedding = nn.Embedding(output_size, hidden_size)
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.out = nn.Linear(hidden_size, output_size)
#         self.softmax = nn.LogSoftmax(dim=1)

#     def forward(self, input, hidden):
#         output = self.embedding(input).view(1, 1, -1)
#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)
#         output = self.softmax(self.out(output[0]))
#         return output, hidden

#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)
# #         return torch.zeros(1, 1, self.hidden_size)

In [95]:
# class AttnDecoderRNN(nn.Module):
#     def __init__(self, hidden_size, output_size, dropout_p = 0.1, max_length = text_length_threshold):
#         super(AttnDecoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.dropout_p = dropout_p
#         self.max_length = max_length

#         self.embedding = nn.Embedding(self.output_size, self.hidden_size)
#         self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
#         self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
#         self.dropout = nn.Dropout(self.dropout_p)
#         self.gru = nn.GRU(self.hidden_size, self.hidden_size)
#         self.out = nn.Linear(self.hidden_size, self.output_size)

#     def forward(self, input, hidden, encoder_outputs):
#         embedded = self.embedding(input).view(1, 1, -1)
#         embedded = self.dropout(embedded)

#         attn_weights = F.softmax(
#             self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
#         attn_applied = torch.bmm(attn_weights.unsqueeze(0),
#                                  encoder_outputs.unsqueeze(0))

#         output = torch.cat((embedded[0], attn_applied[0]), 1)
#         output = self.attn_combine(output).unsqueeze(0)

#         output = F.relu(output)
#         output, hidden = self.gru(output, hidden)

#         output = F.log_softmax(self.out(output[0]), dim=1)
#         return output, hidden, attn_weights

#     def initHidden(self):
#         return torch.zeros(1, 1, self.hidden_size, device=device)

In [134]:
# def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=text_length_threshold):
#     encoder_hidden = encoder.initHidden()

#     encoder_optimizer.zero_grad()
#     decoder_optimizer.zero_grad()

#     input_length = input_tensor.size(0)
#     target_length = target_tensor.size(0)
# #     input_length = len(input_tensor)
# #     target_length = len(target_tensor)
                       
#     encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

#     loss = 0
#     #print('bbbbbbb-->>> input length', input_length)

#     for ei in range(input_length):
#         encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
#         #print("priting before error")
#         #print(encoder_output.size())
#         #print(encoder_outputs.size())
#         temp = encoder_output[0, 0]
#         #print(temp)
#         encoder_outputs[ei] = temp

#     decoder_input = torch.tensor([[SOS_token]], device=device)

#     decoder_hidden = encoder_hidden

#     #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
#     #Without teacher forcing: use its own predictions as the next input

#     #print('aaaaa-->>>')

#     for di in range(target_length):
#         decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)

#         topv, topi = decoder_output.topk(1)
      
#         decoder_input = topi.squeeze().detach()  # detach from history as input
      
#         loss += criterion(decoder_output, target_tensor[di])
      
#         if decoder_input.item() == EOS_token:
#             break

#     loss.backward()

#     encoder_optimizer.step()
#     decoder_optimizer.step()

#     return loss.item() / target_length

In [157]:
# def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
#     start = time.time()
#     plot_losses = []
#     print_loss_total = 0  # Reset every print_every
#     plot_loss_total = 0  # Reset every plot_every

#     encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
#     decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
# #     training_pairs = [tensorsFromPair(random.choice(pairs))
# #                       for i in range(n_iters)]
#     criterion = nn.NLLLoss()

#     for i in range(1, n_iters + 1):
# #         training_pair = training_pairs[iter - 1]
# #         input_tensor = training_pair[0]
# #         target_tensor = training_pair[1]
#         input_tensor = text_train[i]
#         target_tensor = headline_train[i]

#         loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
#         print_loss_total += loss
#         plot_loss_total += loss

#         if i % print_every == 0:
#             print_loss_avg = print_loss_total / print_every
#             print_loss_total = 0
# #             print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
# #                                          iter, iter / n_iters * 100, print_loss_avg))

#         if i % plot_every == 0:
#             plot_loss_avg = plot_loss_total / plot_every
#             plot_losses.append(plot_loss_avg)
#             plot_loss_total = 0

#     showPlot(plot_losses)

In [158]:
# hidden_size = 256
# encoder1 = EncoderRNN(len(text_train_str), hidden_size).to(device)
# attn_decoder1 = AttnDecoderRNN(hidden_size, len(headline_train_str), dropout_p=0.1).to(device)

# trainIters(encoder1, attn_decoder1, 150, print_every=30)

TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not str

In [ ]:
# class EncoderRNN(nn.Module):

#     def __init__(self, embed_size, hidden_size, bidi=True, *, rnn_drop: float=0):
#         super(EncoderRNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_directions = 2 if bidi else 1
#         self.gru = nn.GRU(embed_size, hidden_size, bidirectional=bidi, dropout=rnn_drop)

#     def forward(self, embedded, hidden, input_lengths=None):
#     """
#     :param embedded: (src seq len, batch size, embed size)
#     :param hidden: (num directions, batch size, encoder hidden size)
#     :param input_lengths: list containing the non-padded length of each sequence in this batch;
#                           if set, we use `PackedSequence` to skip the PAD inputs and leave the
#                           corresponding encoder states as zeros
#     :return: (src seq len, batch size, hidden size * num directions = decoder hidden size)
#     Perform multi-step encoding.
#     """
#         if input_lengths is not None:
#             embedded = pack_padded_sequence(embedded, input_lengths)

#         output, hidden = self.gru(embedded, hidden)

#         if input_lengths is not None:
#             output, _ = pad_packed_sequence(output)

#         if self.num_directions > 1:
#           # hidden: (num directions, batch, hidden) => (1, batch, hidden * 2)
#             batch_size = hidden.size(1)
#             hidden = hidden.transpose(0, 1).contiguous().view(1, batch_size, self.hidden_size * self.num_directions)
#         return output, hidden

#     def init_hidden(self, batch_size):
#         return torch.zeros(self.num_directions, batch_size, self.hidden_size, device=DEVICE)


### BERT 

In [ ]:
# # Pytorch library for BERT
# import torch
# from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# # Load pre-trained model tokenizer (vocabulary)
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# # Tokenized input
# text = "Who was Jim Henson ? Jim Henson was a puppeteer"
# tokenized_text = tokenizer.tokenize(text)

# # Mask a token that we will try to predict back with `BertForMaskedLM`
# masked_index = 6
# tokenized_text[masked_index] = '[MASK]'
# assert tokenized_text == ['who', 'was', 'jim', 'henson', '?', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer']

# # Convert token to vocabulary indices
# indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
# segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

# # Convert inputs to PyTorch tensors
# tokens_tensor = torch.tensor([indexed_tokens])
# segments_tensors = torch.tensor([segments_ids])


## TextRank

In [ ]:
# import pandas as pd
# import numpy as np
# import pickle
# from sklearn.metrics.pairwise import cosine_similarity
# import networkx as nx
# from datetime import datetime as dt

# t1 = dt.now()
# print(t1)

# output_file = 'train_stats_dict_processed_extr_final_5000_inc_pagerank.pickle' 
# input_file = 'train_stats_dict_processed_extr_final_5000_.pickle' 
# data = pd.read_pickle(input_file )

# #Select sentence embeddings only and match to doc label
# df_embed = data['df_X'].loc[:,'Sent_BERT_D_0': 'Sent_BERT_D_767']
# df_doc_label = pd.DataFrame(data['Xy_doc_label_array'],columns=['doc_label'])
# df = pd.concat([df_doc_label, df_embed], axis=1)

# #loop through articles (docs)
# pagerank_scores_list=[]
# error_list = []
# doc_num = np.max(data['Xy_doc_label_array']) 
# for j in range(doc_num+1):
    
#     #calculate cosine similiarity matrix 
#     df_doc = df [df.doc_label == j].iloc[:,2:]
#     n = df_doc.shape[0]
#     cos_matrix = cosine_similarity(df_doc, df_doc)
#     f = np.vectorize(lambda x: 0 if x == 1 else 1)
#     not_eye = f(np.eye(n,n))
#     cos_matrix = cos_matrix * not_eye
    
#     #Convert to nx graph
#     graph = nx.from_numpy_array(cos_matrix)
    
#     #Calculate sentence scores and record error docs
#     try:
#         scores_arr = np.array(list(nx.pagerank(graph, max_iter=500).values()))
#     except:
#         scores_arr = np.nan
#         error_list.append(j)
   
#     pagerank_scores_list.append(scores_arr)
    
# pagerank_scores_arr = np.array(pagerank_scores_list)

# #store in primary dictionary
# data.update({'textrank_scores_arr_per_doc':pagerank_scores_arr })

# #save to pickle
# with open(output_file, 'wb') as handle:                                     
#     pickle.dump(data, handle)

# t2=dt.now()
# print(t2)
# print(t2-t1)

# #runtime 4mins50sec for 5000 docs / 29 errors